<a href="https://colab.research.google.com/github/thecognifly/AIYVisionKit_Utils/blob/master/Darth_Vader_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using transfer learning to detect my Darth Vader clock

So, in this notebook I show how I trained an object detector for the [Google AYI Vision Kit](https://aiyprojects.withgoogle.com/vision). This hardware is already old, but I like it because it is a hat for the RPI Zero W that, in addition to the neural net accelerator it has PWM and ADC, and it doesn't use a lot of power (less than 2W in my tests while running the object detector). Moreover, it weighs less than 30g! However, its age brings some problems because Tensorflow kept evolving since they launched the AIY Vision Kit, so I could only train the object detection model that fits the kit using Tensorflow 1.15.2. After training, it will be necessary to export (freeze) the model before you can compile (convert) it for the Google AIY Bonnet, but that step only works on Tensorflow 1.14, lol!

BTW, I'm not trying to train a super-duper Darth Vader detector (actually, the latest checkpoints are overfitting because the dataset I collected doesn't have a lot of diversity and I didn't stop training when I should...), these notebooks are only simple step-by-step instructions to allow you (and myself in the future) to train different object detectors that are capable of running on the Google AIY Vision Kit.

In fact, this model is tiny, so it may be useful for running somewhere else... maybe directly in a RPI?!?

In [1]:
!pip install git+git://github.com/ricardodeazambuja/colab_utils.git --upgrade

  Cloning git://github.com/ricardodeazambuja/colab_utils.git to /tmp/pip-req-build-0gy74x8m
  Running command git clone -q git://github.com/ricardodeazambuja/colab_utils.git /tmp/pip-req-build-0gy74x8m
  Created wheel for colab-utils: filename=colab_utils-0.2-cp36-none-any.whl size=19461 sha256=3392de27fb1f102358be039901e1f105cffbc9538072f1258a90a9134ccf1443
  Stored in directory: /tmp/pip-ephem-wheel-cache-_rq62zeb/wheels/21/75/32/38aeb76b2424385f43eae9fd28c98e084308f4f6d9cb0a4f97
Successfully built colab-utils


In [2]:
from colab_utils import copy2clipboard, webcam2numpy, labelImage, imshow

In [3]:
import os

# The lines below will allow me to easily switch between colab and local notebooks
MYNOTEBOOKPATH = '/content/' # inside the notebook
os.environ['MYNOTEBOOKPATH'] = MYNOTEBOOKPATH # in the shell commands

!echo "MYNOTEBOOKPATH="$MYNOTEBOOKPATH
print(f"MYNOTEBOOKPATH={MYNOTEBOOKPATH}")

MYNOTEBOOKPATH=/content/
MYNOTEBOOKPATH=/content/


In [4]:
from IPython import get_ipython

is_colab = False

try:
  #%tensorflow_version 1.x
  get_ipython().magic('tensorflow_version 1.x')
  is_colab = True
except:
  print("not in colab...")

TensorFlow 1.x selected.


If the current version available within collab doesn't work anymore, just install the 1.15.2. First, uninstall the one installed:  
`!pip uninstall tensorflow -y`  
After that, if things haven't changed, you can install using pip:  
`!pip install tensorflow==1.15.2 > /dev/null`  
One problem: the pip one will not be optimized and that is bad for training...

Luckly, when I was creating this notebook, the current version was 1.15.2

In [5]:
!python -c "import tensorflow as tf; print(tf.__version__)" # python version for the shell commands
# FutureWarning seems to be a Numpy problem when using tf < 1.15: https://stackoverflow.com/a/58546467

1.15.2


In [6]:
!uname -a
!lsb_release -a
!nvidia-smi

Linux 9d86d16cbafc 4.19.112+ #1 SMP Thu Jul 23 08:00:38 PDT 2020 x86_64 x86_64 x86_64 GNU/Linux
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic
Mon Dec 21 13:41:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |  

In [7]:
!cd $MYNOTEBOOKPATH && git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 32.54 MiB/s, done.
Resolving deltas: 100% (575/575), done.


The cell below will prints lots of warnings, but don't worry... they will not bite you back later ;)

In [8]:
!cd $MYNOTEBOOKPATH/cocoapi/PythonAPI && make > /dev/null #redirects stdout to the null hole, but stderr will show...

/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
       ^~~
../common/maskApi.c:46:49: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
                                                 ^
../common/maskApi.c: In function ‘rleFrPoly’:
../common/maskApi.c:166:3: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
   ^~~
../common/maskApi.c:166:54: note: ...this

**Important Step: I only tested it using my modified version of the TensorFlow Object Detection API. I fixed some silly bugs, but I have no idea whether or not the official latest version will work. So, we need to clone my fork!**

In [9]:
!cd $MYNOTEBOOKPATH && git clone https://github.com/ricardodeazambuja/models.git && cd $MYNOTEBOOKPATH/models && git checkout laptop_rtx

Cloning into 'models'...
remote: Enumerating objects: 34169, done.
remote: Total 34169 (delta 0), reused 0 (delta 0), pack-reused 34169
Receiving objects: 100% (34169/34169), 518.62 MiB | 37.47 MiB/s, done.
Resolving deltas: 100% (22328/22328), done.
Checking out files: 100% (2502/2502), done.
Branch 'laptop_rtx' set up to track remote branch 'laptop_rtx' from 'origin'.
Switched to a new branch 'laptop_rtx'


In [11]:
!cp -r $MYNOTEBOOKPATH/cocoapi/PythonAPI/pycocotools $MYNOTEBOOKPATH/models/research/

In [12]:
os.chdir(f"{MYNOTEBOOKPATH}models/research/") # for changing the current working directory everywhere
                                              # the normal cd (without !) would do the same job... but it doesn't accept variables

In [13]:
# yup, I keep checking things...
pwd

'/content/models/research'

In [14]:
!protoc object_detection/protos/*.proto --python_out=.

In [15]:
!echo $PYTHONPATH

/tensorflow-1.15.2/python3.6:/env/python


If you change environmental variables using something like `!export MYVAR = 'something'` the changes will only affect that shell and all the commands will need to be executed in tandem, e.g.: `!export MYVAR = 'something' && echo $MYVAR`.  
However, there are some caveats. `PYTHONPATH` needs to be set **BEFORE** the interpreter is started, but in a notebook the interpreter must be started to use the notebooks itself, so the changes made to the `PYTHONPATH` will not be seen by the interpreter. A solution in this situation is to change the system path instead.

In [16]:
import os
import sys

RESEARCHPATH = os.path.join(MYNOTEBOOKPATH,"models/research")
if RESEARCHPATH not in os.environ['PYTHONPATH']:
  print(f"Changing the $PYTHONPATH from:\n{os.environ['PYTHONPATH']}")
  os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + ":" + RESEARCHPATH + ":" + os.path.join(RESEARCHPATH, "slim")
  print(f"To:\n{os.environ['PYTHONPATH']}")

if RESEARCHPATH not in "".join(sys.path):
  print(f"Changing the system path from:\n{sys.path}")
  sys.path.append(RESEARCHPATH)
  sys.path.append(os.path.join(RESEARCHPATH, "slim"))
  print(f"To:\n{sys.path}")

Changing the $PYTHONPATH from:
/tensorflow-1.15.2/python3.6:/env/python
To:
/tensorflow-1.15.2/python3.6:/env/python:/content/models/research:/content/models/research/slim
Changing the system path from:
['/tensorflow-1.15.2/python3.6', '', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']
To:
['/tensorflow-1.15.2/python3.6', '', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/models/research', '/content/models/research/slim']


The cell below, if it works as it should (that means, you didn't forget to execute anything above...), it will give you 'OK' at the end.

In [17]:
!python $MYNOTEBOOKPATH/models/research/object_detection/builders/model_builder_test.py 

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_wi

I strongly suggest you to have a look at this to learn more about the tfrecord: https://www.tensorflow.org/tutorials/load_data/tfrecord

In [18]:
#
# Just one "tiny" detail here: it will mount your gdrive, so if you mess with it
# you may destroy other data you saved there ;)
#
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


I hope it's clear the dataset used here was created [converting the videos saved in the RPI](https://github.com/thecognifly/AIYVisionKit_Utils/blob/master/Darth_Vader_Convert_RPI_Videos_2_PNG_images.ipynb) and [annotating them](https://github.com/thecognifly/AIYVisionKit_Utils/blob/master/Darth_Vader_Annotate_Images.ipynb).

In [19]:
DATASETPATH = "/content/drive/My Drive/Datasets/DarthVader/"

### Generate the TFRecords (this doesn't need to be repeated if it's already available...)

The object detection API used expects the dataset to be following the Pascal VOC standards. And that's what we do in the next cells.

In [20]:
os.chdir(DATASETPATH)

In [21]:
import json

filename = "/content/drive/My Drive/Datasets/DarthVader/annotations.json"
with open(filename, 'r') as f:
  read_annotations = json.load(f)

In [22]:
# As always, I will test if it worked...
read_annotations['darthvader1.png'][0][1]

'darthvader'

In [23]:
from PIL import Image

# !pip install natsort
from natsort import natsorted # https://natsort.readthedocs.io/en/master/

# 
# This step will convert the annotations to the format used by Pascal VOC
#
with open('darthvader_dataset.csv', 'w') as f:
  f.write("filename,width,height,class,xmin,ymin,xmax,ymax\n")
  for filename in natsorted(read_annotations.keys()):
    img = Image.open(filename)
    w,h = img.size
    box = read_annotations[filename][0][0]
    obj_class = read_annotations[filename][0][1]
    xmin = int(min((box[0],box[0]+box[2]))*w)
    ymin = int(min((box[1],box[1]+box[3]))*h)
    xmax = int(max((box[0],box[0]+box[2]))*w)
    ymax = int(max((box[1],box[1]+box[3]))*h)
    f.write(",".join([filename,str(int(w)),str(int(h)),obj_class,str(xmin),str(ymin),str(xmax),str(ymax)])+'\n')

In [25]:
# testing stuff again...
!head darthvader_dataset.csv

filename,width,height,class,xmin,ymin,xmax,ymax
darthvader1.png,256,256,darthvader,119,147,246,229
darthvader2.png,256,256,darthvader,107,153,240,237
darthvader3.png,256,256,darthvader,122,184,255,255
darthvader4.png,256,256,darthvader,106,134,250,252
darthvader5.png,256,256,darthvader,62,87,246,255
darthvader6.png,256,256,darthvader,67,147,198,253
darthvader7.png,256,256,darthvader,56,72,195,249
darthvader8.png,256,256,darthvader,87,71,230,249
darthvader9.png,256,256,darthvader,119,79,239,241


In [27]:
# Confirming the number of samples is correct by counting the lines (wc -l)
!cat darthvader_dataset.csv | wc -l

95


In [32]:
import numpy as np

rdn = np.random.RandomState(seed=999) # to make sure the sets will not change 
                                      # when this cell is executed again

#
# Couldn't we simply use np.random.seed? Yes and no. 
# If you use np.random.seed with other libraries, the somewhere else
# something may change the seed. The method above is bullet proof ;)
#

n_samples = 95-1
indices = np.arange(n_samples)
rdn.shuffle(indices)

with open("darthvader_dataset.csv", 'r') as f:
  header = f.readline()
  all_lines = f.readlines()
  with open("darthvader_dataset_train.csv", 'w') as ftrain:
    ftrain.write(header)
    with open("darthvader_dataset_test.csv", 'w') as ftest:
      ftest.write(header)
      for li,line in enumerate(all_lines):
        if li in indices[:int(n_samples*0.9)]:
          ftrain.write(line)
        else:
          ftest.write(line)

In [33]:
# More testing... (remember, the first line is the header, so it's 10+84=>94)
!wc -l darthvader_dataset_test.csv
!wc -l darthvader_dataset_train.csv

11 darthvader_dataset_test.csv
85 darthvader_dataset_train.csv


It's always a good idea to have a look at the first lines at least...

In [34]:
!head darthvader_dataset_test.csv

filename,width,height,class,xmin,ymin,xmax,ymax
darthvader9.png,256,256,darthvader,119,79,239,241
darthvader18.png,256,256,darthvader,106,60,175,174
darthvader35.png,256,256,darthvader,83,154,155,255
darthvader53.png,256,256,darthvader,86,183,212,255
darthvader57.png,256,256,darthvader,81,139,198,253
darthvader71.png,256,256,darthvader,51,122,190,253
darthvader80.png,256,256,darthvader,108,62,222,236
darthvader98.png,256,256,darthvader,16,27,173,254
darthvader101.png,256,256,darthvader,89,26,246,252


In [35]:
!head darthvader_dataset_train.csv

filename,width,height,class,xmin,ymin,xmax,ymax
darthvader1.png,256,256,darthvader,119,147,246,229
darthvader2.png,256,256,darthvader,107,153,240,237
darthvader3.png,256,256,darthvader,122,184,255,255
darthvader4.png,256,256,darthvader,106,134,250,252
darthvader5.png,256,256,darthvader,62,87,246,255
darthvader6.png,256,256,darthvader,67,147,198,253
darthvader7.png,256,256,darthvader,56,72,195,249
darthvader8.png,256,256,darthvader,87,71,230,249
darthvader10.png,256,256,darthvader,67,114,159,242


By using the magic `%%file darthvader_object-detection.pbtxt`, the cell below will write its content to the file `darthvader_object-detection.pbtxt`

In [36]:
%%file darthvader_object-detection.pbtxt
item {
  id: 1
  name: 'darthvader'
}

Overwriting darthvader_object-detection.pbtxt


In [38]:
# If you don't believe me...
!cat darthvader_object-detection.pbtxt

item {
  id: 1
  name: 'darthvader'
}

Finally, we will create the TFRecords

In [39]:
#
# Heavily inspired by (copied from) https://github.com/datitran/raccoon_dataset
#
import os
import io
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util

def readCSV(filename, debug=False):
  """Read a CSV file and return a bunch of dictionaries"""
  with open(filename, 'r') as f:
    header = f.readline().strip('\n').split(',')
    if debug: print(header)
    samples = []

    for l in f.readlines():
      if l:
        current_sample = l.strip('\n').split(',')
        if debug: print(current_sample)
        samples.append(dict(zip(header, current_sample)))
  return samples

def getClassesIds(filename):
  """Read a pbtxt file to return a dictionary with classes / ids"""
  ids = []
  classes = []
  with open(filename, 'r') as f:
    for l in f.readlines():
      if 'id' in l:
        ids.append(int(l.strip('\n').split(':')[-1]))
      if 'name' in l:
        classes.append(''.join(c for c in l.split(':')[-1].strip('\n') if c not in " '"))
  return dict(zip(classes,ids))


def create_tf_example(sample, path, class_id):
  with tf.io.gfile.GFile(os.path.join(path, f"{sample['filename']}"), 'rb') as fid:
    encoded_jpg = fid.read()
  image = Image.open(io.BytesIO(encoded_jpg))
  width, height = image.size

  filename = sample['filename'].encode('utf8')
  image_format = image.format.lower().encode('utf-8')
  xmins = [float(sample['xmin']) / width]
  xmaxs = [float(sample['xmax']) / width]
  ymins = [float(sample['ymin']) / height]
  ymaxs = [float(sample['ymax']) / height]
  classes_text = [sample['class'].encode('utf8')]
  classes = [class_id[sample['class']]]

  tf_example = tf.train.Example(features=tf.train.Features(feature={
    'image/height': dataset_util.int64_feature(height),
    'image/width': dataset_util.int64_feature(width),
    'image/filename': dataset_util.bytes_feature(filename),
    'image/source_id': dataset_util.bytes_feature(filename),
    'image/encoded': dataset_util.bytes_feature(encoded_jpg),
    'image/format': dataset_util.bytes_feature(image_format),
    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
    'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
  return tf_example


def generateTFRecordFiles(OUTPUT_PATH, IMG_PATH, CSV_PATH, PBTXT_PATH):
  class_id = getClassesIds(PBTXT_PATH)
  with tf.python_io.TFRecordWriter(OUTPUT_PATH) as writer:
    path = os.path.join(IMG_PATH)
    samples = readCSV(CSV_PATH)
    for sample in samples:
      tf_example = create_tf_example(sample, path, class_id)
      writer.write(tf_example.SerializeToString())

  print(f'Successfully created the TFRecords: {OUTPUT_PATH}')

In [40]:
IMG_PATH = DATASETPATH
PBTXT_PATH = os.path.join(DATASETPATH,"darthvader_object-detection.pbtxt")

OUTPUT_PATH = os.path.join(DATASETPATH,"test.record")
CSV_PATH = os.path.join(DATASETPATH,"darthvader_dataset_test.csv")
generateTFRecordFiles(OUTPUT_PATH, IMG_PATH, CSV_PATH, PBTXT_PATH)

OUTPUT_PATH = os.path.join(DATASETPATH,"train.record")
CSV_PATH = os.path.join(DATASETPATH,"darthvader_dataset_train.csv")
generateTFRecordFiles(OUTPUT_PATH, IMG_PATH, CSV_PATH, PBTXT_PATH)

Successfully created the TFRecords: /content/drive/My Drive/Datasets/DarthVader/test.record
Successfully created the TFRecords: /content/drive/My Drive/Datasets/DarthVader/train.record


Now that our data is inside those two TFRecords files, it's necessary to set the config file for the Object Detection API:

In [41]:
%%file ssd_mobilenetv1_256x256_AIY_VOC2012.config

# Nice explanations can be found here:
# https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
#
# Or directly from the source:
# https://github.com/tensorflow/models/blob/master/research/object_detection/utils/config_util.py
#

# Embedded SSD with Mobilenet v1 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 1 # if you have more classes (I have only Darth Vader clock...)
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 5
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    # This model already has a image resizer (256x256) embedded
    image_resizer {
      fixed_shape_resizer {
        height: 256
        width: 256
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'embedded_ssd_mobilenet_v1' # the embedded version is the only one that worked for me
      min_depth: 16
      depth_multiplier: 0.125 # anything bigger than this will be too big for the AIY Vision Kit
      override_base_feature_extractor_hyperparams: true
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 0
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    normalize_loc_loss_by_codesize: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 3 # changed from 100 to 3
        max_total_detections: 3 # changed from 100 to 3
      }
      score_converter: SIGMOID
    }
  }
}
train_config: {
  batch_size: 32 # this number allowed me to train without memory problems. If you are having problems, make it smaller
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }
  fine_tune_checkpoint_type: 'detection'
  # The directory below needs to match your own system. This one reflects a directory Checkpoints inside my google drive
  # where there's another directory called VOC2012_Trained with the checkpoint
  fine_tune_checkpoint: "/content/drive/My Drive/Checkpoints/VOC2012_Trained/model.ckpt-870894"
  # the pre-trained model came from: https://github.com/google/aiyprojects-raspbian/issues/314#issuecomment-391414207
  # https://drive.google.com/file/d/1_MeZ8kvmpNibPZvSJGnwKNRATeuyxNtu/view?usp=sharing
  from_detection_checkpoint: true #DEPRECATED????

  # I went crazy with the data augmentation...
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_vertical_flip {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }
  data_augmentation_options {
    random_adjust_saturation {
    }
  }
  data_augmentation_options {
    random_rotation90 {
    }
  }
  data_augmentation_options {
    random_adjust_brightness {
    }
  }
  data_augmentation_options {
    random_distort_color {
    }
  } 
  data_augmentation_options {
    random_jitter_boxes {
    }
  }
  data_augmentation_options {
    random_image_scale {
    }
  }
  data_augmentation_options {
    random_black_patches {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    # If you saved your stuff in a different directory, change it here and below (and above!)
    input_path: "/content/drive/My Drive/Datasets/DarthVader/train.record"
  }
  label_map_path: "/content/drive/My Drive/Datasets/DarthVader/darthvader_object-detection.pbtxt"
}

eval_config: {
  # (Optional): Uncomment the line below if you installed the Coco evaluation tools
  # and you want to also run evaluation (if you are following my notebooks, you did install)
  metrics_set: "coco_detection_metrics"
  # https://github.com/tensorflow/models/issues/2225#issuecomment-325596658
  use_moving_averages: true
  # (Optional): Set this to the number of images in your train set
  # if you want to also run evaluation
  num_examples: 11
  num_visualizations: 11
  eval_interval_secs: 60
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  # max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/Datasets/DarthVader/test.record"
  }
  label_map_path: "/content/drive/My Drive/Datasets/DarthVader/darthvader_object-detection.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting ssd_mobilenetv1_256x256_AIY_VOC2012.config


### Training will start soon...

In [42]:
os.chdir(f"{MYNOTEBOOKPATH}models/research/")

**Where do I set the number of steps or time that takes until an event file is created?**
- eventsout...
The only thing I noticed is: it will save when I stop it and that looks like some cache algorithm.

The nice thing about this setup is that you can stop and restart and it will automatically pick up from the last checkpoint if you pass the same folder (there's a text file called "checkpoint" that keeps track of that and you can even edit it) 

In [ ]:
!mkdir "/content/drive/My Drive/Checkpoints/DarthVader/"

Training really starts below. Currently, it's saving 50 checkpoints (save_checkpoints_steps) and it will print a lot of stuff on the output. Jupyter notebooks slow down if a cell becomes too big, therefore you can use the `> /dev/null` to avoid that.

Even without Tensorboard, you can check how your training is doing by reading the `INFO:tensorflow:loss = ????, step = ???? (???? sec)` lines in the output. The first time takes a while.

If you want to use Tensorboard, I prepared a notebook exactly for that:  
https://github.com/thecognifly/AIYVisionKit_Utils/blob/master/Tensorboard_from_GDrive.ipynb

In [ ]:
!MODELDIR="/content/drive/My Drive/Checkpoints/DarthVader/" && \
python object_detection/model_main.py \
    --pipeline_config_path="/content/drive/My Drive/Datasets/DarthVader/ssd_mobilenetv1_256x256_AIY_VOC2012.config" \
    --model_dir="${MODELDIR}" \
    --num_train_steps=30000 \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr \
    --save_checkpoints_steps=50#> /dev/null